# AutoGen 基本サンプル

このコードサンプルでは、[AutoGen](https://aka.ms/ai-agents/autogen) AI フレームワークを使用して基本的なエージェントを作成します。

このサンプルの目的は、さまざまなエージェントパターンを実装する際に、後続のコードサンプルで使用する手順を示すことです。

## 必要なPythonパッケージのインポート

In [1]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## クライアントの作成

このサンプルでは、LLMへのアクセスに[GitHub Models](https://aka.ms/ai-agents-beginners/github-models)を使用します。

`model`は`gpt-4o-mini`として定義されています。GitHub Models マーケットプレイスで利用可能な別のモデルに変更して、異なる結果を確認してみてください。

簡単なテストとして、シンプルなプロンプト「フランスの首都は何ですか？」を実行してみます。

In [6]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4.1",
    endpoint="https://models.inference.ai.azure.com",
    # モデルで認証するには、GitHubの設定でパーソナルアクセストークン（PAT）を生成する必要があります。
    # PATトークンの作成方法については、以下の手順に従ってください: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="フランスの首都は何ですか？", source="user")])
print(result)

c:\Users\Mamoru Kuroda\AppData\Local\Programs\Python\Python313\Lib\site-packages\autogen_ext\models\azure\_azure_ai_client.py:305: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(config["model_info"])


finish_reason='stop' content='フランスの首都はパリ（Paris）です。' usage=RequestUsage(prompt_tokens=18, completion_tokens=15) cached=False logprobs=None thought=None


## エージェントの定義

`client`を設定し、正常に動作することを確認したので、`AssistantAgent`を作成しましょう。各エージェントには以下を割り当てることができます：

**name** - マルチエージェントフローで参照する際に有用な短縮名。

**model_client** - 前のステップで作成したクライアント。

**tools** - エージェントがタスクを完了するために使用できる利用可能なツール。

**system_message** - LLMのタスク、動作、およびトーンを定義するメタプロンプト。

システムメッセージを変更して、LLMがどのように応答するかを確認できます。`tools`については、レッスン#4で説明します。


In [7]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="あなたは尊敬する会社の上司です。あなたの仕事は、ユーザーに仕事のアドバイスを与えることです。",
)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000002D5C78CE7B0>


## エージェントの実行

以下の関数がエージェントを実行します。`on_message`メソッドを使用して、エージェントの状態を新しいメッセージで更新します。

この場合、「素晴らしい晴れた休暇を計画してください」というユーザーからの新しいメッセージで状態を更新します。

メッセージ内容を変更して、LLMがどのように異なる応答をするかを確認できます。

In [8]:
from IPython.display import display, HTML


async def assistant_run():
    # クエリを定義
    user_query = "新しい組織やチーム、同僚と仕事をすることになった。英語が初めてでどうすればいいかわからなくて不安。"

    # HTML出力の構築を開始
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>ユーザー:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # エージェント応答の実行
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # エージェントの応答をHTMLに追加
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>アシスタント:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # フォーマットされたHTMLを表示
    display(HTML(html_output))

# 関数を実行
await assistant_run()